# Install and call library

In [ ]:
# Install gurobipy library
!pip install gurobipy

In [ ]:
# Call gurobipy library
import gurobipy as gp
from gurobipy import *

# Call pandas library
import pandas as pd

# Setup sets and parameters

In [ ]:
# Sets
I = 20 # Set of workforce (Labor 1, Labor 2, ..., Labor 20)
J = 7 # Set of weekday (Monday, Tuesday, ..., Sunday)
K = 3 # Set of shift (Morning, Afternoon, Evening)

# Parameters
demand = [
    [6,4,3], ## Monday demand,
    [5,5,5],## Tuesday demand,
    [6,4,4], ## Wednesday demand,
    [5,5,5], ## Thursday demand,
    [5,5,3], ## Friday demand,
    [6,5,4], ## Saturday demand,
    [5,5,4] ## Sunday demand
]

# Setup model

In [ ]:
# Create a gurobipy model
model = gp.Model("staff_assignment")

# Decision variable
x = model.addVars(I, J, K, vtype=GRB.BINARY, name="Assigning staff i to day j at shift k")

# Objective function
model.setObjective(gp.quicksum(x[i, j, k] for i in range(I) for j in range(J) for k in range(K)), GRB.MINIMIZE)

# Constraints

## For each shift k in each day j, the number of labor to be assigned must fulfill the demand
model.addConstrs((gp.quicksum(x[i, j, k] for i in range(I)) == demand[j][k] for j in range(J) for k in range(K)), name="demand fulfill")

## For each labor i in each day j, the maximum number of shift he/she can work is 1
model.addConstrs((gp.quicksum(x[i, j, k] for k in range(K)) <= 1 for i in range(I) for j in range(J)), name="max 1 shift a day")

## For each labor i, the maximum day and shift he/she can work is 5
model.addConstrs((gp.quicksum(x[i, j, k] for j in range(J) for k in range(K)) <= 5 for i in range(I)), name="only work 5 days and 5 shifts a week")

## For each labor i in each 2 days j-1 and j, if he/she works on evening shift (k = 3) today, he/she cannot work on the morning shift (k = 1) tomorrow
model.addConstrs((x[i, j, 2] + x[i, j+1, 0] <= 1 for i in range(I) for j in range(J-1)), name="if work on shift 3 today then no work on shift 1 tomorrow")

Restricted license - for non-production use only - expires 2026-11-23


{(0, 0): <gurobi.Constr *Awaiting Model Update*>,
 (0, 1): <gurobi.Constr *Awaiting Model Update*>,
 (0, 2): <gurobi.Constr *Awaiting Model Update*>,
 (0, 3): <gurobi.Constr *Awaiting Model Update*>,
 (0, 4): <gurobi.Constr *Awaiting Model Update*>,
 (0, 5): <gurobi.Constr *Awaiting Model Update*>,
 (1, 0): <gurobi.Constr *Awaiting Model Update*>,
 (1, 1): <gurobi.Constr *Awaiting Model Update*>,
 (1, 2): <gurobi.Constr *Awaiting Model Update*>,
 (1, 3): <gurobi.Constr *Awaiting Model Update*>,
 (1, 4): <gurobi.Constr *Awaiting Model Update*>,
 (1, 5): <gurobi.Constr *Awaiting Model Update*>,
 (2, 0): <gurobi.Constr *Awaiting Model Update*>,
 (2, 1): <gurobi.Constr *Awaiting Model Update*>,
 (2, 2): <gurobi.Constr *Awaiting Model Update*>,
 (2, 3): <gurobi.Constr *Awaiting Model Update*>,
 (2, 4): <gurobi.Constr *Awaiting Model Update*>,
 (2, 5): <gurobi.Constr *Awaiting Model Update*>,
 (3, 0): <gurobi.Constr *Awaiting Model Update*>,
 (3, 1): <gurobi.Constr *Awaiting Model Update*>,


In [ ]:
# Optimize model
model.optimize()

Gurobi Optimizer version 12.0.0 build v12.0.0rc1 (linux64 - "Ubuntu 22.04.3 LTS")

CPU model: Intel(R) Xeon(R) CPU @ 2.20GHz, instruction set [SSE2|AVX|AVX2]
Thread count: 1 physical cores, 2 logical processors, using up to 2 threads

Optimize a model with 301 rows, 420 columns and 1500 nonzeros
Model fingerprint: 0xd9d16d0a
Variable types: 0 continuous, 420 integer (420 binary)
Coefficient statistics:
  Matrix range     [1e+00, 1e+00]
  Objective range  [1e+00, 1e+00]
  Bounds range     [1e+00, 1e+00]
  RHS range        [1e+00, 6e+00]
Found heuristic solution: objective 99.0000000
Presolve time: 0.00s
Presolved: 301 rows, 420 columns, 1500 nonzeros
Variable types: 0 continuous, 420 integer (420 binary)

Explored 0 nodes (0 simplex iterations) in 0.02 seconds (0.00 work units)
Thread count was 2 (of 2 available processors)

Solution count 1: 99 

Optimal solution found (tolerance 1.00e-04)
Best objective 9.900000000000e+01, best bound 9.900000000000e+01, gap 0.0000%


In [ ]:
if model.status == GRB.OPTIMAL:
    # Prepare data for the table
    table_data = []

    for i in range(I):
        for j in range(J):
            shifts = [k + 1 for k in range(K) if x[i, j, k].x > 1e-6]
            if shifts:
                for k in shifts:
                    table_data.append({
                        'Staff': i + 1,  # Staff numbers starting from 1
                        'Day': j + 1,    # Days starting from 1
                        'Shift': k,      # Shifts starting from 1
                        'Status': 'Assigned'
                    })
            else:
                # No shifts assigned on this day, so staff is on leave
                table_data.append({
                    'Staff': i + 1,
                    'Day': j + 1,
                    'Shift': 'N/A',
                    'Status': 'Leave'
                })

    # Create DataFrame
    df = pd.DataFrame(table_data)

    # Print the DataFrame
    print(df)

    # Optionally, save to a CSV file
    df.to_csv('staff_assignment.csv', index=False)

else:
    print("No optimal solution found.")

     Staff  Day Shift    Status
0        1    1   N/A     Leave
1        1    2     1  Assigned
2        1    3     2  Assigned
3        1    4     2  Assigned
4        1    5   N/A     Leave
..     ...  ...   ...       ...
135     20    3   N/A     Leave
136     20    4   N/A     Leave
137     20    5     3  Assigned
138     20    6     3  Assigned
139     20    7     2  Assigned

[140 rows x 4 columns]
